In [1]:
import torch
import numpy as np
import random
import wandb

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from ucimlrepo import fetch_ucirepo

from noise_schedulers import LinearScheduler, PowerScheduler
from tempering import TemperingLearningRegression
from models import MLP

seed = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print ("Device: ", device)
print ("Seed: ", seed)

# Set random seed for reproducibility
# This ensures consistent results across different runs
torch.manual_seed(seed)  # Set seed for PyTorch
np.random.seed(seed)     # Set seed for NumPy
random.seed(seed)        # Set seed for Python's random module

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Device:  cuda
Seed:  42


wandb: Currently logged in as: zhan3692. Use `wandb login --relogin` to force relogin


True

In [9]:
fetch_ucirepo(id=275).data.features

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
0,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,2012-12-31,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642
17375,2012-12-31,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642
17376,2012-12-31,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642
17377,2012-12-31,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343


In [2]:
# fetch dataset 
wine_quality = fetch_ucirepo(id=186) 
  
# data (as pandas dataframes) 
X = wine_quality.data.features.to_numpy()
y = wine_quality.data.targets.to_numpy()

# train test split
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

# standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# convert to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)

X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

# summary of the dataset
print ("X_train shape: ", X_train.shape)
print ("y_train shape: ", y_train.shape)
print ("X_test shape: ", X_test.shape)
print ("y_test shape: ", y_test.shape)

N = X_train.shape[0]

X_train shape:  torch.Size([5197, 11])
y_train shape:  torch.Size([5197, 1])
X_test shape:  torch.Size([1300, 11])
y_test shape:  torch.Size([1300, 1])


In [3]:
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'T': {
        'values': [50, 100, 200, 500]
        },
    'init_sigma': {
        'values': [2.0, 1.0, 0.1]
    },
    'lr': {
        'values': [1e-4, 1e-5, 1e-6]
    },
    'n': {
        'values': [0.1, 0.3, 0.5, 0.7]
    },
    'm': {
        'values': [0.1, 0.3, 0.5, 0.7]
    },
    'MC_steps': {
        'values': [50, 100, 200, 400, 1000]
    },
    'burn_in_fraction': {
        'values': [0.1, 0.3, 0.5]
    },
    'zeta': {
        'values': [1.0, 0.1, 0.01]
    },
    'tau': {
        'value': 1.0
    }
}

sweep_config['parameters'] = parameters_dict

sweep_id = wandb.sweep(sweep_config, project="TL-WineQuality")


Create sweep with ID: 6mh2noxb
Sweep URL: https://wandb.ai/zhan3692/TL-WineQuality/sweeps/6mh2noxb


In [4]:
def fit(config=None):

    with wandb.init(config=config):

        config = wandb.config

        noise_scheudler = LinearScheduler(T=config.T, init_sigma=config.init_sigma)
        D = noise_scheudler.transform(y_train)
        sigmas = noise_scheudler.sigmas

        model = MLP(X_train.shape[-1], [16, 16], 1).to(device)

        trainer = TemperingLearningRegression(
            D = D,
            X = X_train,
            model = model,
            sigmas = sigmas,
            tau = config.tau,
            zeta = config.zeta,
            init_lr = config.lr,
            burn_in_fraction=config.burn_in_fraction,
            MC_steps=config.MC_steps,
            n = config.n,
            m = config.m,
            X_test = X_test,
            y_test = y_test,
            logger = wandb,
            progress_bar = False
        )

        trainer.train()

In [ ]:
wandb.agent(sweep_id, fit, count=5)